In [124]:
import catboost
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import time

In [128]:
df = pd.read_csv('Clean_Data.csv', sep=';')
df_no_cat = df.drop(columns=['student_id','student_email','group_id','group_title','prep_email','date_first_l','feat_group_format','feat_payment_type','feat_payment_place'])

In [129]:
df.describe()

,student_id,group_id,feat_class,feat_group_weeks,feat_group_size,feat_attendance,feat_HW,feat_HW_right,feat_theory_read,feat_TK,feat_LB,feat_tickets_we_need,feat_tickets_mb_term,feat_term_perc,feat_good_agr_before,target,feat_prep_attendence,feat_prep_HW,feat_prep_OpenLessons
count,6.733800e+04,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000,67338.000000
mean,5.819781e+06,40180.076435,10.248849,29.343268,21.898096,0.887237,0.618251,0.486862,0.438003,0.539958,2.826547,0.353990,0.023167,17.719212,0.938979,0.134293,0.929856,0.737299,4.658664
std,4.484174e+06,13695.909825,0.903591,6.640665,25.418138,0.208232,0.299286,0.257425,0.320549,0.308143,1.982354,1.174556,0.204566,6.715806,0.239370,0.340969,0.037516,0.102348,0.705510
min,2.742000e+03,13917.000000,9.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.620000,0.000000,0.000000
25%,3.075420e+05,28162.000000,9.000000,29.000000,12.000000,0.880000,0.400892,0.291120,0.150000,0.290000,1.000000,0.000000,0.000000,13.471503,1.000000,0.000000,0.909316,0.680000,4.150000
50%,9.000006e+06,36769.000000,11.000000,33.000000,16.000000,0.980000,0.677750,0.511811,0.390000,0.530000,3.000000,0.000000,0.000000,17.254902,1.000000,0.000000,0.930000,0.742044,4.658664
75%,9.622290e+06,51729.000000,11.000000,33.000000,20.000000,1.000000,0.877086,0.695808,0.710000,0.800000,5.000000,0.000000,0.000000,22.236663,1.000000,0.000000,0.956094,0.805000,5.000000
max,1.142207e+07,72244.000000,11.000000,35.000000,194.000000,1.000000,1.000000,1.000000,1.000000,1.000000,27.000000,24.000000,7.000000,100.000000,1.000000,1.000000,1.000000,1.000000,6.000000


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67338 entries, 0 to 67337
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   student_id             67338 non-null  int64  
 1   student_email          67338 non-null  object 
 2   group_id               67338 non-null  int64  
 3   group_title            67338 non-null  object 
 4   prep_email             67338 non-null  object 
 5   date_first_l           67338 non-null  object 
 6   feat_class             67338 non-null  int64  
 7   feat_group_format      67338 non-null  object 
 8   feat_group_weeks       67338 non-null  int64  
 9   feat_group_size        67338 non-null  int64  
 10  feat_attendance        67338 non-null  float64
 11  feat_HW                67338 non-null  float64
 12  feat_HW_right          67338 non-null  float64
 13  feat_theory_read       67338 non-null  float64
 14  feat_TK                67338 non-null  float64
 15  fe

In [131]:
encoded_ohe_df = df_no_cat
ohe = OneHotEncoder(sparse_output=False, dtype=int)

ohe_df_1 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_payment_type']]), columns=ohe.get_feature_names_out(['feat_payment_type']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_1], axis=1) # В том случае, если хотим добавить закодированные данные
ohe = OneHotEncoder(sparse_output=False, dtype=int)
ohe_df_2 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_payment_place']]), columns=ohe.get_feature_names_out(['feat_payment_place']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_2], axis=1) # В том случае, если хотим добавить закодированные данные
ohe = OneHotEncoder(sparse_output=False, dtype=int)
ohe_df_3 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_group_format']]), columns=ohe.get_feature_names_out(['feat_group_format']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_3], axis=1) # В том случае, если хотим добавить закодированные данные

In [133]:
# Выделяем строку корреляции с целевой переменной
target_corr = corr_matrix['target'].abs().sort_values(ascending=False)

# Выбираем признаки с корреляцией выше порога (например, 0.3)
strong_features = target_corr[target_corr > 0.05].index.tolist()

print(f"Лучшие признаки: {strong_features}")

Лучшие признаки: ['target', 'feat_good_agr_before', 'feat_attendance', 'feat_LB', 'feat_HW', 'feat_HW_right', 'feat_theory_read', 'feat_TK', 'feat_payment_type_Позтапная', 'feat_payment_type_Единовременная', 'feat_tickets_mb_term', 'feat_term_perc', 'feat_payment_place_Навигатор', 'feat_prep_HW']


In [135]:
df_few = encoded_ohe_df[strong_features]
X = df_few.drop('target', axis = 1)
y = df_few['target']

In [136]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np

## 1. Catboost model with unbalanced data

In [158]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [159]:
start = time.time()
model1 = CatBoostClassifier(iterations=100, random_state=42,  learning_rate=0.2, verbose=0,  task_type='GPU',  boosting_type='Ordered')
model1.fit(X1_train, y1_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 1.62 cекунд


In [160]:
y1_pred = model1.predict(X1_test)
print(classification_report(y1_test, y1_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96     11659
           1       0.84      0.56      0.67      1809

    accuracy                           0.93     13468
   macro avg       0.89      0.77      0.82     13468
weighted avg       0.92      0.93      0.92     13468



In [161]:
print(f'f-1 score: {round(cross_val_score(model1, X, y, scoring='f1').mean(),3)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')

f-1 score: 0.669 
Время кроссвалидации: 11.44 секунд


## 2. Catboost model + oversampling on train dataset

In [179]:
from imblearn.over_sampling import RandomOverSampler

In [196]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [197]:
ros = RandomOverSampler()
X_over_train, y_over_train = ros.fit_resample(X2_train, y2_train)

In [198]:
start = time.time() 
model_over = CatBoostClassifier(iterations=100, random_state=42,  learning_rate=0.2, verbose=0,  task_type='GPU',  boosting_type='Ordered')
model_over.fit(X_over_train, y_over_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 1.53 cекунд


In [199]:
y_over_pred = model_over.predict(X2_test)
print(classification_report(y2_test, y_over_pred))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92     11659
           1       0.51      0.81      0.63      1809

    accuracy                           0.87     13468
   macro avg       0.74      0.84      0.77     13468
weighted avg       0.91      0.87      0.88     13468



## 3. Catboost model + undersampling on dataset

In [109]:
from imblearn.under_sampling import RandomUnderSampler

In [207]:
rus = RandomUnderSampler()
X_under, y_under = rus.fit_resample(X, y)

X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)

In [211]:
start = time.time() 
model_under = CatBoostClassifier(iterations=100, random_state=42,  learning_rate=0.2, verbose=0,  task_type='GPU',  boosting_type='Ordered')
model_under.fit(X_under_train, y_under_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 1.36 cекунд


In [212]:
y_under_pred = model_under.predict(X_under_test)
print(classification_report(y_under_test, y_under_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.83      1801
           1       0.85      0.80      0.82      1817

    accuracy                           0.83      3618
   macro avg       0.83      0.83      0.83      3618
weighted avg       0.83      0.83      0.83      3618



In [213]:
print(f'f-1 score: {round(cross_val_score(model_under, X_under, y_under, scoring='f1').mean(),3)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')

f-1 score: 0.828 
Время кроссвалидации: 13.83 секунд


## 4. Catboost model + SMOTE

In [226]:
from imblearn.over_sampling import SMOTE

In [227]:
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X1_train, y1_train)

In [228]:
start = time.time() 
cat_smote = CatBoostClassifier(iterations=100, random_state=42,  learning_rate=0.2, verbose=0,  task_type='GPU',  boosting_type='Ordered')
cat_smote.fit(X_smote, y_smote)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 1.57 cекунд


In [220]:
y_smote_pred = cat_smote.predict(X1_test)
print(classification_report(y1_test, y_smote_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93     11659
           1       0.55      0.74      0.63      1809

    accuracy                           0.88     13468
   macro avg       0.76      0.82      0.78     13468
weighted avg       0.90      0.88      0.89     13468



## 4. Catboost model + ADASYN

In [229]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train, y_train)

In [230]:
start = time.time() 
cat_adasyn = CatBoostClassifier(iterations=100, random_state=42,  learning_rate=0.2, verbose=0,  task_type='GPU',  boosting_type='Ordered')
cat_adasyn.fit(X_adasyn, y_adasyn)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 1.5 cекунд


In [ ]:
y_adasyn_pred = cat_adasyn.predict(X_test)
print(classification_report(y_test, y_adasyn_pred))